## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
os.chdir("../Weather_Database")
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hithadhoo,MV,-0.6000,73.0833,79.95,76,98,4.59,overcast clouds
1,1,Sinnamary,GF,5.3833,-52.9500,79.43,83,84,12.01,broken clouds
2,2,Thompson,CA,55.7435,-97.8558,69.96,68,75,8.05,broken clouds
3,3,Hobart,AU,-42.8794,147.3294,53.26,72,40,1.99,scattered clouds
4,4,Wamba,NG,8.9333,8.6000,70.59,98,89,3.00,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hithadhoo,MV,-0.6000,73.0833,79.95,76,98,4.59,overcast clouds
1,1,Sinnamary,GF,5.3833,-52.9500,79.43,83,84,12.01,broken clouds
7,7,Kapaa,US,22.0752,-159.3190,84.18,68,47,4.00,scattered clouds
11,11,Monywa,MM,22.1167,95.1333,80.42,82,100,7.76,overcast clouds
16,16,Salalah,OM,17.0151,54.0924,76.12,91,91,6.11,overcast clouds
17,17,Albany,US,42.6001,-73.9662,78.76,73,22,5.41,few clouds
19,19,Amarante Do Maranhao,BR,-5.5667,-46.7422,79.56,39,35,3.00,scattered clouds
27,27,Bethel,US,41.3712,-73.4140,76.08,82,93,3.80,overcast clouds
33,33,Kahului,US,20.8947,-156.4700,85.96,68,13,8.50,few clouds
34,34,Adrar,MR,20.5022,-10.0711,86.95,20,9,10.80,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                240
City                   240
Country                240
Lat                    240
Lng                    240
Max Temp               240
Humidity               240
Cloudiness             240
Wind Speed             240
Current Description    240
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hithadhoo,MV,79.95,overcast clouds,-0.6000,73.0833,
1,Sinnamary,GF,79.43,broken clouds,5.3833,-52.9500,
7,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,
11,Monywa,MM,80.42,overcast clouds,22.1167,95.1333,
16,Salalah,OM,76.12,overcast clouds,17.0151,54.0924,
17,Albany,US,78.76,few clouds,42.6001,-73.9662,
19,Amarante Do Maranhao,BR,79.56,scattered clouds,-5.5667,-46.7422,
27,Bethel,US,76.08,overcast clouds,41.3712,-73.4140,
33,Kahului,US,85.96,few clouds,20.8947,-156.4700,
34,Adrar,MR,86.95,clear sky,20.5022,-10.0711,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)


In [10]:
# 8a. Create the output File (CSV)
os.chdir("../Vacation_Search")
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))